# Assignment 4.1: Tilt Alarm System

## Part A4.1

### Buzzer sensor module used: KY006 (Passive) or KY012- (Active)

#### Include placeholder

In [ ]:
import time
from pynq.lib import Pmod_IO
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
import socket
import multiprocessing

In [ ]:
FAILURE = -1
SUCCESS = 0
SERVER_FREQ = 2500

In [ ]:
#creat Pmod_IO object to be used for server. Wired to + of buzzer sensor to PMODB
gpio_pin_1 = Pmod_IO(base.PMODB,1,'out')
#create Pmod_IO object to be used for client. Wired to + of buzzer sensor to PMODA
gpio_pin_2 = Pmod_IO(base.PMODA,1,'out')

In [ ]:
#store these pins in a dictionary to easily refrence based on server client
pins = {
    'server'   : gpio_pin_1, 
    'client'   : gpio_pin_2,
}

In [ ]:
# definition to turn buzzer off
def turn_buzzer_off(index):
    pins[index].write(0)

In [ ]:
# def function to test buzzer
def turn_buzzer_on(index):
    pins[index].write(1)

In [ ]:
# def function to turn buzzer on for particular frequency
def turn_alarm_on(freq, index, play_time):
    if freq <= 0:
        print("frequency should be > 0")
        return FAILURE
    if index not in pins:
        print("index passed does not exist {}".format(index))
        return FAILURE
    period = round(1.0/freq,6)
    cycles = round(play_time/period)
    while cycles:
        turn_buzzer_on(index)
        time.sleep((1/(2*freq)))
        turn_buzzer_off(index)
        time.sleep((1/(2*freq)))
        cycles -=1
    
    return SUCCESS

In [ ]:
freq = 25000 #Hz
index = 'server'
count = 0
turn_alarm_on(freq, index, 0.5)
count +=1

In [ ]:
from enum import Enum
class Button_State(Enum):
    Start      = 0
    Connect    = 1
    Alarm      = 2
    Disconnect = 3

In [ ]:
BUTTON_START =      0b01
BUTTON_ALARM =      0b10
BUTTON_DISCONNECT = 0b100

In [ ]:
def server():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 1: Bind the socket to the pynq board <CLIENT-IP> at port <LISTENING-PORT>
    # bind the socket to a public host, and a well-known port
    sock.bind(("127.0.0.1", 33334))
    
    #Always be a listener
    sock.listen(5)
    cond = False;
    while True:
        print ('Waiting connection from client')
        # 2: Accept connections
        clientsocket, addr = sock.accept()
        print ('Got connection from', addr )
        
        while True:
        # 3: Receive bytes from the connection 
            msg = clientsocket.recv(1024).decode()
            print ("Message received {}\n".format(msg))
        
            if msg == "alarm":
                print ("ServermMessage {}\n".format(msg))
                cond = True
         
            if msg == "disconnect":
                print ("Server Message {}\n".format(msg))
                cond = False
                break
           
            if cond == True:
                print ("Server Buzzer On\n")
                cond = False
                turn_alarm_on(SERVER_FREQ, 'server', 0.3)

In [ ]:
def client(btns):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect(('127.0.0.1', 33334))
    Button_cur_State = Button_State.Connect
    
    cond = True
    
    while True:
        time.sleep(0.01)
        if btns.read() == BUTTON_ALARM:
            if Button_cur_State != Button_State.Alarm:
                Button_cur_State = Button_State.Alarm
            print("Client: alarm Button state = {}\n".format(Button_cur_State))
            sock.send("alarm".encode())
            time.sleep(0.5)#let server process the message
        if btns.read() == BUTTON_DISCONNECT:
            if Button_cur_State != Button_State.Disconnect:
                print("Client: disconnect")
                Button_cur_State = Button_State.Disconnect
                print("Client: Disconnect Button state = {}\n".format(Button_cur_State))
                sock.send("disconnect".encode())
                time.sleep(0.5)#let server process the message
                break
    sock.close()

In [ ]:
procs = [] # a future list of all our processes


In [ ]:
import threading
import os
def main():
    btns = base.btns_gpio
    Button_cur_State = Button_State.Disconnect
    #create a asyncio loop event     
    # Launch process1 on CPU0
    p1_start = time.time()
    p1 = multiprocessing.Process(target=server, args=()) 
    os.system("taskset -p -c {} {}".format(0, p1.pid)) 
    p1.start() # start the process
    
    # Launch process2 on CPU1
    while True:
        time.sleep(0.1)
        if btns.read() == BUTTON_START:
            if Button_cur_State == Button_State.Disconnect:
                Button_cur_State = Button_State.Start
                print("Creating client")
                p2_start = time.time()
                p2 = multiprocessing.Process(target=client, args=(btns,)) 
                os.system("taskset -p -c {} {}".format(1, p2.pid))
                p2.start() # start the process
                procs.append(p2)
                time.sleep(0.5)
        if btns.read() == BUTTON_DISCONNECT:
            if Button_cur_State != Button_State.Disconnect:
                p2 = procs.pop()
                p2.join() # wait for process2 to finish
                Button_cur_State = Button_State.Disconnect
                print('Process 2 with name, {}, is finished'.format(p2.name))
                time.sleep(0.5)

In [ ]:
if __name__ == '__main__':
    main()